In [1]:
import numpy as np
import sys
import os
#import tensorflow as tf

import io
from gensim.models import word2vec

from io import StringIO
from pprint import pprint

from nltk.tokenize import word_tokenize
import nltk

In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import types
from IPython import get_ipython #for import notebook
from nbformat import read #for import notebook
from IPython.core.interactiveshell import InteractiveShell #for import notebook

In [3]:
def find_notebook(fullname, path=None): #for import notebook
    """find a notebook, given its fully qualified name and an optional path

    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path

In [4]:
class NotebookLoader(object): #for import notebook
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path

    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)

        print ("importing Jupyter notebook from %s" % path)

        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = read(f, 4)


        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod

        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__

        try:
          for cell in nb.cells:
            if cell.cell_type == 'code':
                # transform the input to executable Python
                code = self.shell.input_transformer_manager.transform_cell(cell.source)
                # run the code in themodule
                exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod
class NotebookFinder(object):
    """Module finder that locates Jupyter Notebooks"""
    def __init__(self):
        self.loaders = {}

    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return

        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)

        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]

In [5]:
sys.meta_path.append(NotebookFinder())

In [6]:
import torch
import torch.nn as nn
from torch import optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [7]:
import torch.utils.data
from torchvision import transforms, utils
from torch.utils.data import DataLoader

In [8]:
import pandas as pd

In [9]:
class dialogueDataset(torch.utils.data.Dataset):


    def __init__(self, npfile, npfile_t, emotion_file, action_file, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        
        self.data_index = np.load(npfile)#"./indexed_dataset/i_dt.npy"
        self.data_index_t= np.load(npfile_t)#"./indexed_dataset/t_dt.npy"
        
        self.data_emotion= np.load(emotion_file)
        self.data_action= np.load(action_file)
        
        
        self.transform = transform

        
    def __len__(self):
        return len(self.data_emotion)

    
    def __getitem__(self, idx):
        
        
        dialogue = self.data_index[idx]
        dialogue_t = self.data_index_t[idx]
        dialogue_data = [dialogue,dialogue_t]
        
        d_emotion = self.data_emotion[idx]
        d_action = self.data_action[idx]
        dialogue_index = [d_emotion,d_action]
        
        
        sample = {'dialogue_1': dialogue, 'label_1': d_emotion,
                  'dialogue_2': dialogue_t, 'label_2': d_action }

        if self.transform:
            sample = self.transform(sample)

        return sample


In [10]:
transformed_train_dataset = dialogueDataset( npfile='/home/jongsu/Desktop/intention_extraction/indexed_dataset/i_dt.npy', 
                                            npfile_t = '/home/jongsu/Desktop/intention_extraction/indexed_dataset/t_dt.npy',
                                            emotion_file='/home/jongsu/Desktop/intention_extraction/indexed_dataset/d_emo.npy',
                                            action_file='/home/jongsu/Desktop/intention_extraction/indexed_dataset/d_act.npy')


transformed_test_dataset = dialogueDataset( npfile='/home/jongsu/Desktop/intention_extraction/indexed_dataset/i_dt_test.npy', 
                                           npfile_t = '/home/jongsu/Desktop/intention_extraction/indexed_dataset/t_dt_test.npy',
                                           emotion_file='/home/jongsu/Desktop/intention_extraction/indexed_dataset/d_emo_test.npy',
                                           action_file='/home/jongsu/Desktop/intention_extraction/indexed_dataset/d_act_test.npy')


In [11]:
for i in range(len(transformed_train_dataset)):
    sample = transformed_train_dataset[i]

    print("\n",sample['dialogue_1'])
    print("\n",sample['dialogue_2'])
    
    print("\n",sample['label_1'])
    print("\n",sample['label_2'])
    
    if i == 3:
        break




 [[1561, 1, 1089, 1, 102, 33, 74, 14, 7, 229, 3510, 218, 354, 4], [32, 39, 16, 10, 4904, 43, 10, 68, 44, 60, 14, 81, 2579, 0], [29, 21, 3, 225, 4, 41, 35, 93, 149, 6, 1314, 0], [71, 3, 68, 38, 40, 4, 2, 64, 0, 41, 35, 51, 106, 149, 1473, 8, 2582, 1735, 0, 1409, 177, 61, 4], [2, 325, 3, 19, 6800, 57, 654, 30, 21, 4, 2, 64, 207, 24, 1492, 36, 224, 0], [2, 746, 7, 547, 144, 6, 5, 1215, 230, 30, 23, 348, 4963, 8, 273, 56, 12, 81, 358, 0], [97, 7, 60, 228, 0, 2, 318, 476, 8, 2153, 364, 47, 46, 6, 348, 'pingpong.Perhaps', 30, 23, 106, 7, 'foursome', 28, 99, 0], [609, 133, 6, 20, 17, 140, 88, 19, 1180, 1, 30, 121, 250, 99, 6, 47, 1383, 28, 'us.That', 10, 771, 645, 8, 449, 1, 78, 0], ['Good.Let', 161, 34, 47, 92, 0], [219, 55, 0], []]

 [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1,

In [12]:
from torch.utils.data import Dataset, DataLoader

dataloader = DataLoader(transformed_train_dataset, batch_size=4,
                        shuffle=True, num_workers=4)


In [13]:
for i_batch, sample_batched in enumerate(dataloader):
    
    print("\n",i_batch)
    print("\n",sample_batched['dialogue_1']) 
    print("\n",sample_batched['dialogue_2'])

TypeError: Traceback (most recent call last):
  File "/home/jongsu/.pyenv/versions/anaconda3-4.2.0/lib/python3.5/site-packages/torch/utils/data/dataloader.py", line 57, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/jongsu/.pyenv/versions/anaconda3-4.2.0/lib/python3.5/site-packages/torch/utils/data/dataloader.py", line 135, in default_collate
    return {key: default_collate([d[key] for d in batch]) for key in batch[0]}
  File "/home/jongsu/.pyenv/versions/anaconda3-4.2.0/lib/python3.5/site-packages/torch/utils/data/dataloader.py", line 135, in <dictcomp>
    return {key: default_collate([d[key] for d in batch]) for key in batch[0]}
  File "/home/jongsu/.pyenv/versions/anaconda3-4.2.0/lib/python3.5/site-packages/torch/utils/data/dataloader.py", line 138, in default_collate
    return [default_collate(samples) for samples in transposed]
  File "/home/jongsu/.pyenv/versions/anaconda3-4.2.0/lib/python3.5/site-packages/torch/utils/data/dataloade